# Retries with Reflection

Sometimes Agents will not succeed in their task. This might be e.g. if their reasoning was wrong (i.e., they made the wrong plan) or inefficient (i.e., they are taking too long to get to an answer). In this case, we can use the re-try method Reflection (from https://arxiv.org/abs/2303.11366).

With Reflection, the Agent will think about why their plan failed and what it can do better next time. This reflection, along with the previous trial(s) will be available to the Agent in the next trial. This enables the Agent to solve harder problems.

This notebook will shows you how to use Reflection.

First, we import libraries and load an LLM and tools; just like we would for an Agent without Reflection.

In [1]:
from langchain.agents import load_tools, initialize_agent, AgentType
from langchain.agents.reflexion.react import ReactReflector
from langchain.llms import OpenAI
from langchain.utilities import WikipediaAPIWrapper

In [ ]:
llm = OpenAI(temperature=0)

In [ ]:
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

Next, we create a `Reflector` object. The default implementation is `ReactReflector`.

In [ ]:
reflector = ReactReflector.from_llm(
    llm=llm,
    max_iterations_per_trial=5
    # =10.0,
    # we    max_action_repetition=2
)

Reflection only kicks in, wenn the Reflector determines that the current trial failed. For this, we have to pass in conditions that specify when a trial failed.
These can be:
- `max_iterations_per_trial`: how many steps can the Agent take in a trial?
- `max_execution_time_per_trial`: how much take can the Agent use in a trial?
- `max_action_repetition`: how often is the Agent allowed to suggest the exact same Actions?

Finally, let's initialize an agent with the `tools`, the `llm`, the type of `Agent` we want to use and our `Reflector`.

In [6]:
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    reflector=reflector,
    verbose=True
    # we could also pass: 
    # 
)

Now let's test it out!

In [13]:
question = "Author David Chanoff has collaborated with a U.S. Navy admiral who served as the ambassador to the United Kingdom under which President?"
agent.run(question)



> Entering new AgentExecutor chain...
 I need to find out who Leo DiCaprio's girlfriend is and then calculate her age raised to the 0.43 power.
Action: Search
Action Input: "Leo DiCaprio girlfriend"
Observation: Camila Morrone
Thought: I need to find out Camila Morrone's age
Action: Search
Action Input: "Camila Morrone age"
Observation: 25 years
Thought: I need to calculate 25 raised to the 0.43 power
Action: Calculator
Action Input: 25^0.43
Observation: Answer: 3.991298452658078

Thought: I now know the final answer
Final Answer: Camila Morrone is Leo DiCaprio's girlfriend and her current age raised to the 0.43 power is 3.991298452658078.

> Finished chain.


"Camila Morrone is Leo DiCaprio's girlfriend and her current age raised to the 0.43 power is 3.991298452658078."